<a href="https://colab.research.google.com/github/JunhanBai/Pratice/blob/main/Prompt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
SUMMARY_PROMPT = '''\
You are a precise summarizer. Given the text below, produce a concise summary in 3–5 sentences.
Requirements: capture the main claim, the key supporting points, and any concrete figures or timelines.
Avoid opinions, avoid filler, and do not invent facts.

Text:
{text}
'''

CRITIQUE_PROMPT = '''\
You are a careful reviewer. Critique the summary using the rubric below, then suggest concrete edits.
Rubric (0–2 each): Relevance, Coherence, Accuracy, Completeness, Clarity.
- Identify missing points, redundancies, vague wording, and any potential hallucinations.
- Provide bullet-pointed suggestions; be specific and actionable.

Original Text (reference):
{text}

Model Summary (to critique):
{summary}
'''

IMPROVE_PROMPT = '''\
You are a senior editor. Rewrite the summary by applying the critique items below.
Constraints: 3–5 sentences; factual; concrete; neutral tone.
If a fact isn't in the original text, omit it.

Original Text (reference):
{text}

Critique items:
{critique}

Produce ONLY the improved summary.
'''
print('Loaded SUMMARY_PROMPT, CRITIQUE_PROMPT, IMPROVE_PROMPT')

## Deterministic helpers (simulate LLM behavior offline)

In [ ]:
import re, textwrap
from collections import Counter
from typing import Dict

def _sentences(s: str):
    parts = re.split(r'(?<=[.!?])\s+', s.strip())
    return [p.strip() for p in parts if p.strip()]

def mock_summarize(text: str) -> str:
    # Naive: take top frequent content words and stitch 3 short sentences.
    tokens = re.findall(r"[a-z0-9']+", text.lower())
    stops = set('the a an of and to in for on with at by from is are was were be been being this that it as or if but'.split())
    words = [t for t in tokens if t not in stops and len(t) > 2]
    freq = [w for w,_ in Counter(words).most_common(6)]
    sents = _sentences(text)
    lead = sents[0] if sents else ''
    tail = sents[1] if len(sents) > 1 else ''
    gist = ', '.join(freq[:4]) if freq else 'key points'
    out = [f"The text discusses {gist}.", lead[:120], tail[:120]]
    return ' '.join([o for o in out if o])

def mock_critique(text: str, summary: str) -> str:
    # Heuristic checks: coverage (keywords), brevity, and potential hallucination markers.
    toks_text = set(re.findall(r"[a-z0-9']+", text.lower()))
    toks_sum = set(re.findall(r"[a-z0-9']+", summary.lower()))
    coverage = len(toks_sum & toks_text) / (len(toks_sum) + 1e-9)
    too_long = len(summary.split()) > max(60, len(text.split())//3)
    made_up = any(w not in toks_text for w in ['guarantee','always','never']) and False  # placeholder example
    notes = []
    # Rubric scoring (rough, for demo)
    relevance = 2 if coverage > 0.6 else (1 if coverage > 0.4 else 0)
    coherence = 2 if summary.count('.') >= 2 else 1
    accuracy = 2 if not made_up else 0
    completeness = 1
    clarity = 2 if not too_long else 1
    notes.append(f"Scores — Relevance:{relevance}/2 Coherence:{coherence}/2 Accuracy:{accuracy}/2 Completeness:{completeness}/2 Clarity:{clarity}/2")
    if too_long:
        notes.append('Edit for brevity: target 3–5 concise sentences.')
    # Suggest adding any top nouns present in text but absent in summary
    common_in_text = [w for w,_ in Counter(re.findall(r'[a-z0-9']+', text.lower())).most_common(10)]
    missing = [w for w in common_in_text if w not in toks_sum][:3]
    if missing:
        notes.append('Add missing specifics: ' + ', '.join(missing))
    return '\n'.join(notes)

def mock_improve(text: str, summary: str, critique: str) -> str:
    # Apply critique heuristics: shorten if long; append missing keywords.
    sents = _sentences(summary)
    sents = sents[:5]
    body = ' '.join(sents)
    # Add up to 2 suggested keywords
    m = re.search(r'Add missing specifics:\s*(.+)', critique)
    if m:
        adds = ','.split() if False else [w.strip() for w in m.group(1).split(',')][:2]
        extra = ' '.join(adds)
        if extra:
            body = body + ' ' + f"It also mentions {extra}."
    return body

## Orchestrator: Summary → Critique → Improved Summary

In [ ]:
def run_self_reflection_chain(text: str) -> Dict[str, str]:
    summary = mock_summarize(text)
    critique = mock_critique(text, summary)
    improved = mock_improve(text, summary, critique)
    return {
        'filled_summary_prompt': SUMMARY_PROMPT.format(text=text),
        'model_summary': summary,
        'filled_critique_prompt': CRITIQUE_PROMPT.format(text=text, summary=summary),
        'model_critique': critique,
        'filled_improve_prompt': IMPROVE_PROMPT.format(text=text, critique=critique),
        'improved_summary': improved
    }
print('Chain ready: run_self_reflection_chain(text)')

## Demo

In [ ]:
sample_text = (
    "Our customer support team is updating shipping policies to reduce delays during peak seasons. "
    "Orders may take up to five business days to arrive, and customers will receive proactive notifications. "
    "We’re also adding weekend processing to speed fulfillment for priority orders."
)
out = run_self_reflection_chain(sample_text)
print('--- MODEL SUMMARY ---')
print(out['model_summary'])
print('\n--- CRITIQUE ---')
print(out['model_critique'])
print('\n--- IMPROVED SUMMARY ---')
print(out['improved_summary'])